In [ ]:
!uv pip install -q caldav

In [ ]:
from dotenv import load_dotenv
load_dotenv()


In [8]:
import caldav
from caldav.elements import dav
import os
# Replace these values with the user's actual credentials:
APPLE_ID = os.getenv("APPLE_ID")            # The user's Apple ID email address
APPLE_APP_SPECIFIC_PASSWORD = os.getenv("APPLE_APP_SPECIFIC_PASSWORD")  # The generated app-specific password
CALDAV_URL = "https://caldav.icloud.com/"

def authenticate_to_icloud():
    """
    Connects and authenticates to the user's iCloud calendar via CalDAV.
    
    Returns:
        A caldav.DAVClient instance if authentication is successful.
    Raises:
        Exception if authentication fails.
    """
    try:
        # Create a CalDAV client using the provided credentials
        client = caldav.DAVClient(
            url=CALDAV_URL,
            username=APPLE_ID,
            password=APPLE_APP_SPECIFIC_PASSWORD
        )
        
        # Retrieve the principal (the authenticated user) for verification
        principal = client.principal()
        display_name = principal.get_display_name()
        print("Successfully authenticated as:", display_name)
        return client

    except Exception as e:
        print("Failed to authenticate to iCloud CalDAV:", e)
        raise


In [ ]:
client = authenticate_to_icloud()

In [ ]:
client.principal().calendars()[1].


In [ ]:
# List calendars associated with the accounts
calendars = client.principal().calendars()

print("Found", len(calendars), "calendars:")
for cal in calendars:
    print("Calendar:", cal.name)

In [ ]:
# Show some key calendar attributes and methods
new_calendar = calendars[0]
print("Calendar name:", new_calendar.name)
print("Calendar URL:", new_calendar.url)
print("Calendar ID:", new_calendar.id)

# Show some useful methods
print("\nUseful methods:")
print("- events() - Get all events")
print("- event_by_uid() - Get event by UID") 
print("- date_search() - Search events by date")
print("- save_event() - Save a new event")
print("- delete() - Delete the calendar")

In [ ]:
# Get all events
events = new_calendar.events()
print("Found", len(events), "events:")
for event in events:
    print("Event:", event.summary)
    print("Event start:", event.start)
    print("Event end:", event.end)
    print("Event location:", event.location)
    print("Event description:", event.description)


In [ ]:
# create a new event
from datetime import datetime, timedelta

start = datetime.now()
end = start + timedelta(hours=1)

start_str = start.strftime("%Y%m%dT%H%M%S")
end_str = end.strftime("%Y%m%dT%H%M%S")

ical:str = f"""BEGIN:VCALENDAR
VERSION:2.0
BEGIN:VEVENT
SUMMARY:New Event
DTSTART:{start_str}
DTEND:{end_str}
LOCATION:Home
DESCRIPTION:This is a new event
END:VEVENT
END:VCALENDAR"""

event = new_calendar.save_event(ical)

print("New event created:", event)

In [ ]:
# Create a recurring yearly event
my_event = new_calendar.save_event(
    dtstart=datetime.now(),
    dtend=datetime.now() + timedelta(hours=2),
    summary="Yearly Planning Meeting",
)

print("Created event:", my_event)


In [ ]:
new_calendar.search(comp_class='event')

In [ ]:
### Create a new calendar
try:
    # Get the principal (user) object
    principal = client.principal()
    
    # Create new calendar using make_calendar()
    new_calendar = principal.make_calendar(name="my time table")
    print(f"New calendar 'my time table' created successfully!")
    
    # List available calendars to verify
    calendars = principal.calendars()
    print("\nAvailable calendars:")
    for cal in calendars:
        print(f" - {cal.name}")

except Exception as e:
    print("Error creating calendar:", e)
    raise

In [ ]:
new_calendar.id

In [ ]:
new_calendar.parent

In [72]:
CUSTOM_PROPERTY = "X-SYNCADEMIC-ID"
CUSTOM_PROPERTY_VALUE = "1234567890"

In [ ]:
# Create a new event with custom properties
from datetime import datetime, timedelta

# Create event start and end times
start = datetime.now()
end = start + timedelta(hours=1)

# Format dates in iCalendar format
start_str = start.strftime("%Y%m%dT%H%M%SZ")
end_str = end.strftime("%Y%m%dT%H%M%SZ")

# Create iCalendar string with custom X-property
ical = f"""BEGIN:VCALENDAR
VERSION:2.0
BEGIN:VEVENT
SUMMARY:Event with Custom Property
DTSTART:{start_str}
DTEND:{end_str}
LOCATION:Office
DESCRIPTION:Event with custom X-property
{CUSTOM_PROPERTY}:{CUSTOM_PROPERTY_VALUE}
END:VEVENT
END:VCALENDAR"""

# Save the event
event = new_calendar.save_event(ical)
print("Created event with custom properties:", event)


from datetime import timezone


new_event = new_calendar.save_event(
        # Our custom property:
        **{CUSTOM_PROPERTY: CUSTOM_PROPERTY_VALUE},
        summary="Custom property as keyword argument",
        dtstart=datetime(2025, 1, 10, 9, 0, 0, tzinfo=timezone.utc),
        dtend=datetime(2025, 1, 10, 10, 0, 0, tzinfo=timezone.utc),
    )

# Read back the event and its properties
events = new_calendar.events()
for e in events:
    print("\nEvent details:")
    print(f"Summary: {e.instance.vevent.summary.value}")
    print(f"Start: {e.instance.vevent.dtstart.value}")
    print(f"End: {e.instance.vevent.dtend.value}")
    # print(f"Location: {e.instance.vevent.location.value}")
    # print(f"Description: {e.instance.vevent.description.value}")
    
    # Read custom X-properties
    for prop in e.instance.vevent.contents.values():
        if isinstance(prop, list):
            for p in prop:
                if hasattr(p, 'name') and p.name.startswith('X-'):
                    print(f"{p.name}: {p.value}")

###  Finding events by custom property

In [ ]:
from caldav.elements.cdav import Filter, CompFilter, PropFilter, TextMatch

def find_my_events(calendar: caldav.Calendar) -> list[caldav.Event]:
    """
    Searches the calendar for events that have `X-SYNCADEMIC-ID: 1234567890`.
    """
    # Build a property filter: X-SYNCADEMIC-ID must match "1234567890"
    prop_filter = PropFilter(CUSTOM_PROPERTY)
    prop_filter.append(TextMatch(CUSTOM_PROPERTY_VALUE))
    
    # Inside a VEVENT or VTODO compfilter
    comp_filter = CompFilter("VEVENT")
    comp_filter.append(prop_filter)

    # Wrap everything in a top-level Filter
    filters = Filter()
    filters.append(comp_filter)

    # Now pass that filter to search()
    return calendar.search(filters=filters)

my_events = find_my_events(new_calendar)
print(my_events)


### Getting all events, then filtering by custom property

In [ ]:
all_events = new_calendar.events()  # or use .search()
for event in all_events:
    # event.load()  # ensure event.data is populated
    if CUSTOM_PROPERTY_VALUE in event.data:
        print(event.instance.vevent.summary.value)
    else:
        print("This event does not have the custom property")


### Using categories (not working)


In [ ]:
# Creating:
new_event = new_calendar.save_event(
    categories=["syncademic-1234567890"],
    summary="Demo Event from My Service",
    dtstart=datetime(2025, 1, 10, 9, 0, 0, tzinfo=timezone.utc),
    dtend=datetime(2025, 1, 10, 10, 0, 0, tzinfo=timezone.utc),
)


# Searching for all events with that category, then deleting them:
my_events = new_calendar.search(event=True, category="syncademic-1234567890")
if not my_events:
    print("No events found with that category")
else:
    for evt in my_events:
        print(evt)
        # evt.delete()


### Deleting the calendar

In [ ]:
new_calendar.delete()
print("Calendar deleted")

# list the calendars to check if the new calendar is there
calendars = principal.calendars()
print("\nAvailable calendars:")
for cal in calendars:
    print(f" - {cal.name}")


In [ ]:
with caldav.DAVClient(url=url, username=username, password=password) as client:
    my_principal = client.principal()
    ...